# Analizziamo l'effetto Memoria dei mercati prima mediante semplice correlazione e poi scomponendo i setup:

Prendiamo questa volta una serie intraday:

Costruiamo una serie resampled di period "period":

In [1]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__) # necessaria versione >= 1.9.0

import cufflinks as cf

# Per utilizzo con Notebooks
init_notebook_mode(connected=True)

# Per utilizzo offline
cf.go_offline()


filename = "@GC_60min.txt"
startDate = "20130101"
endDate = "20180810"
period = "D"

startDateParsed = datetime.date(int(startDate[0:4]), int(startDate[4:6]), int(startDate[6:]))
endDateParsed = datetime.date(int(endDate[0:4]), int(endDate[4:6]), int(endDate[6:]))

instrument = pd.read_csv(filename,
                         usecols=['Date','Time','Open','High','Low','Close','Up','Down'], 
                         parse_dates=[['Date', 'Time']])
print("Il file contiene", len(instrument), "record")
instrument.index = instrument['Date_Time']
instrument.drop('Date_Time',axis=1,inplace=True)
instrument['AvgPrice'] = round(instrument.iloc[:,0:4].mean(axis=1),2)
instrument['Range'] = round(instrument['High'] - instrument['Low'],2)
instrument['Body'] = abs(instrument['Open'] - instrument['Close'])
instrument['CloseOpenPerc'] = round((instrument['Close'] - instrument['Open']) / instrument['Open'] * 100,2)
instrument['BodyRangePerc'] = round(instrument['Body']/instrument['Range'] * 100,2)
# Percentuale della barra (all'interno del range) dove si attesta il close
instrument['CloseBarPerc'] = round((instrument['Close'] - instrument['Low']) * 100 / instrument['Range'],2)
instrument['DeltaClosePerc'] = round((instrument['Close'] - instrument['Close'].shift(1)) / instrument['Close'].shift(1) * 100,2)
instrument['DeltaAvgPricePerc'] = round((instrument['AvgPrice'] - instrument['AvgPrice'].shift(1)) / instrument['AvgPrice'].shift(1) * 100,2)

instrument = instrument.loc[startDateParsed:endDateParsed]
#instrument.head(10)

# creazione Pandas DataFrame di lavoro con i soli campi utili
cluster = pd.DataFrame(index=instrument.Open.resample(period).first().dropna().index.copy())
cluster["Open"] = instrument.Open.resample(period).first().dropna()
cluster["High"] = instrument.High.resample(period).max().dropna()
cluster["Low"] = instrument.Low.resample(period).min().dropna()
cluster["Close"] = instrument.Close.resample(period).last().dropna()
cluster['Range'] = round(cluster['High'] - cluster['Low'],2)
cluster['Body'] = abs(cluster['Open'] - cluster['Close'])
cluster['BodyRangePerc'] = round(cluster['Body']/cluster['Range'] * 100,2)
cluster['DeltaClosePerc'] = round((cluster['Close'] - cluster['Close'].shift(1)) / cluster['Close'].shift(1) * 100,2)
cluster['CloseOpenPerc'] = round((cluster['Close'] - cluster['Open']) / cluster['Open'] * 100,2)

cluster.head(10)

#cluster.drop("Open", axis=1, inplace=True)
#cluster.drop("Close", axis=1, inplace=True)

5.4.0


Il file contiene 68540 record


,Open,High,Low,Close,Range,Body,BodyRangePerc,DeltaClosePerc,CloseOpenPerc
Date_Time,,,,,,,,,
2013-01-01,1748.6,1751.1,1743.1,1750.7,8.0,2.1,26.25,NaN,0.12
2013-01-02,1750.6,1767.6,1749.2,1759.0,18.4,8.4,45.65,0.47,0.48
2013-01-03,1759.0,1761.2,1717.8,1724.3,43.4,34.7,79.95,-1.97,-1.97
2013-01-04,1724.3,1731.2,1698.2,1728.5,33.0,4.2,12.73,0.24,0.24
2013-01-06,1728.5,1735.2,1728.1,1732.5,7.1,4.0,56.34,0.23,0.23
2013-01-07,1732.4,1734.3,1714.8,1721.2,19.5,11.2,57.44,-0.65,-0.65
2013-01-08,1721.1,1734.9,1719.2,1730.6,15.7,9.5,60.51,0.55,0.55
2013-01-09,1730.6,1738.2,1723.5,1729.1,14.7,1.5,10.20,-0.09,-0.09
2013-01-10,1729.0,1751.0,1728.0,1744.6,23.0,15.6,67.83,0.90,0.90


Studiamo la correlazione tra i range e i range di N barre indietro:

In [2]:
for i in range(25):
    print(cluster["Range"][i], 
          cluster["Range"].shift(1)[i], 
          cluster["Range"].shift(2)[i],
          cluster["Range"].shift(3)[i],
          cluster["Range"].shift(4)[i],
          cluster["Range"].shift(5)[i])

8.0 nan nan nan nan nan
18.4 8.0 nan nan nan nan
43.4 18.4 8.0 nan nan nan
33.0 43.4 18.4 8.0 nan nan
7.1 33.0 43.4 18.4 8.0 nan
19.5 7.1 33.0 43.4 18.4 8.0
15.7 19.5 7.1 33.0 43.4 18.4
14.7 15.7 19.5 7.1 33.0 43.4
23.0 14.7 15.7 19.5 7.1 33.0
23.8 23.0 14.7 15.7 19.5 7.1
8.4 23.8 23.0 14.7 15.7 19.5
10.9 8.4 23.8 23.0 14.7 15.7
15.2 10.9 8.4 23.8 23.0 14.7
11.7 15.2 10.9 8.4 23.8 23.0
31.4 11.7 15.2 10.9 8.4 23.8
11.8 31.4 11.7 15.2 10.9 8.4
6.3 11.8 31.4 11.7 15.2 10.9
4.2 6.3 11.8 31.4 11.7 15.2
9.6 4.2 6.3 11.8 31.4 11.7
13.7 9.6 4.2 6.3 11.8 31.4
20.0 13.7 9.6 4.2 6.3 11.8
16.6 20.0 13.7 9.6 4.2 6.3
3.5 16.6 20.0 13.7 9.6 4.2
10.6 3.5 16.6 20.0 13.7 9.6
8.6 10.6 3.5 16.6 20.0 13.7


In [3]:
numberofBackCorrelations = 100

correlationsValue = []

for i in range(1,numberofBackCorrelations):
    correlationsValue.append(cluster.Range.corr(cluster.Range.shift(i)) * 100)
    #print(correlationsValue[i-1])

correlationsValue = pd.DataFrame(correlationsValue)
correlationsValue.iplot(kind='bar', color="orange", title="Correlations between Range")

Si nota un aspetto periodico interessante pur con valori bassi di correlazione!

In [4]:
numberofBackCorrelations = 20

correlationsValue = []

for i in range(1,numberofBackCorrelations):
    correlationsValue.append(cluster.DeltaClosePerc.corr(cluster.DeltaClosePerc.shift(i)) * 100)
    print(correlationsValue[i-1])

correlationsValue = pd.DataFrame(correlationsValue)
correlationsValue.iplot(kind='bar', color="blue", title="Correlations between DeltaClosePerc")

0.010223102042620013
1.2328751760334722
-1.6797381783137635
3.924290603461003
-3.231082613834641
4.169305610036131
0.1765362479657997
1.7756824644869946
3.200472242406783
1.1007806408307377
-7.46480822976478
-3.9956508178230097
-3.3362169001677886
1.8625586604302529
0.2411144140080597
-1.2531527813655832
2.2448155918520776
-3.54234742042707
-2.0828330446585666


In [5]:
numberofBackCorrelations = 100

correlationsValue = []

for i in range(1,numberofBackCorrelations):
    correlationsValue.append(cluster.CloseOpenPerc.corr(cluster.CloseOpenPerc.shift(i)) * 100)
    #print(correlationsValue[i-1])

correlationsValue = pd.DataFrame(correlationsValue)
correlationsValue.iplot(kind='bar', color="red", title="Correlations between CloseOpenPerc")

Proviamo a costruire lo stesso studio ma con il rapporto tra corpo e range:

In [6]:
for i in range(25):
    print(cluster["BodyRangePerc"][i], 
          cluster["BodyRangePerc"].shift(1)[i], 
          cluster["BodyRangePerc"].shift(2)[i],
          cluster["BodyRangePerc"].shift(3)[i],
          cluster["BodyRangePerc"].shift(4)[i],
          cluster["BodyRangePerc"].shift(5)[i])

26.25 nan nan nan nan nan
45.65 26.25 nan nan nan nan
79.95 45.65 26.25 nan nan nan
12.73 79.95 45.65 26.25 nan nan
56.34 12.73 79.95 45.65 26.25 nan
57.44 56.34 12.73 79.95 45.65 26.25
60.51 57.44 56.34 12.73 79.95 45.65
10.2 60.51 57.44 56.34 12.73 79.95
67.83 10.2 60.51 57.44 56.34 12.73
43.28 67.83 10.2 60.51 57.44 56.34
69.05 43.28 67.83 10.2 60.51 57.44
56.88 69.05 43.28 67.83 10.2 60.51
69.08 56.88 69.05 43.28 67.83 10.2
33.33 69.08 56.88 69.05 43.28 67.83
31.85 33.33 69.08 56.88 69.05 43.28
28.81 31.85 33.33 69.08 56.88 69.05
42.86 28.81 31.85 33.33 69.08 56.88
66.67 42.86 28.81 31.85 33.33 69.08
8.33 66.67 42.86 28.81 31.85 33.33
81.75 8.33 66.67 42.86 28.81 31.85
70.0 81.75 8.33 66.67 42.86 28.81
54.82 70.0 81.75 8.33 66.67 42.86
37.14 54.82 70.0 81.75 8.33 66.67
10.38 37.14 54.82 70.0 81.75 8.33
91.86 10.38 37.14 54.82 70.0 81.75


In [7]:
numberofBackCorrelations = 100

correlationsValue = []

for i in range(1,numberofBackCorrelations):
    correlationsValue.append(cluster.BodyRangePerc.corr(cluster.BodyRangePerc.shift(i)) * 100)
    #print(correlationsValue[i-1])

correlationsValue = pd.DataFrame(correlationsValue)
correlationsValue.iplot(kind='bar', color="blue", title="Correlations between BodyRangePerc")

In [8]:
numberofBackCorrelations2 = 100

correlationsValue2 = []

for i in range(1,numberofBackCorrelations2):
    correlationsValue2.append(cluster.CloseOpenPerc.corr(cluster.BodyRangePerc.shift(i)))
    #print(correlationsValue2[i-1])

correlationsValue2 = pd.DataFrame(correlationsValue2)
correlationsValue2.iplot(kind='bar', color="blue", title="Multiday Trends")

In [9]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__) # necessaria versione >= 1.9.0

import cufflinks as cf

# Per utilizzo con Notebooks
init_notebook_mode(connected=True)

# Per utilizzo offline
cf.go_offline()


filename = "@GC_60min.txt"
startDate = "20130101"
endDate = "20180810"
period = "D"

startDateParsed = datetime.date(int(startDate[0:4]), int(startDate[4:6]), int(startDate[6:]))
endDateParsed = datetime.date(int(endDate[0:4]), int(endDate[4:6]), int(endDate[6:]))

instrument = pd.read_csv(filename,
                         usecols=['Date','Time','Open','High','Low','Close','Up','Down'], 
                         parse_dates=[['Date', 'Time']])
print("Il file contiene", len(instrument), "record")
instrument.index = instrument['Date_Time']
instrument.drop('Date_Time',axis=1,inplace=True)
instrument['AvgPrice'] = round(instrument.iloc[:,0:4].mean(axis=1),2)
instrument['Range'] = round(instrument['High'] - instrument['Low'],2)
instrument['Body'] = abs(instrument['Open'] - instrument['Close'])
instrument['CloseOpenPerc'] = round((instrument['Close'] - instrument['Open']) / instrument['Open'] * 100,2)
instrument['BodyRangePerc'] = round(instrument['Body']/instrument['Range'] * 100,2)
# Percentuale della barra (all'interno del range) dove si attesta il close
instrument['CloseBarPerc'] = round((instrument['Close'] - instrument['Low']) * 100 / instrument['Range'],2)
instrument['DeltaClosePerc'] = round((instrument['Close'] - instrument['Close'].shift(1)) / instrument['Close'].shift(1) * 100,2)
instrument['DeltaAvgPricePerc'] = round((instrument['AvgPrice'] - instrument['AvgPrice'].shift(1)) / instrument['AvgPrice'].shift(1) * 100,2)

instrument = instrument.loc[startDateParsed:endDateParsed]
#instrument.head(10)

# creazione Pandas DataFrame di lavoro con i soli campi utili
cluster = pd.DataFrame(index=instrument.Open.resample(period).first().dropna().index.copy())
cluster["Open"] = instrument.Open.resample(period).first().dropna()
cluster["High"] = instrument.High.resample(period).max().dropna()
cluster["Low"] = instrument.Low.resample(period).min().dropna()
cluster["Close"] = instrument.Close.resample(period).last().dropna()
cluster['Range'] = round(cluster['High'] - cluster['Low'],2)
cluster['Body'] = abs(cluster['Open'] - cluster['Close'])
cluster['BodyRangePerc'] = round(cluster['Body']/cluster['Range'] * 100,2)
cluster['DeltaClosePerc'] = round((cluster['Close'] - cluster['Close'].shift(1)) / cluster['Close'].shift(1) * 100,2)
cluster['CloseOpenPerc'] = round((cluster['Close'] - cluster['Open']) / cluster['Open'] * 100,2)

cluster.head(10)

#cluster.drop("Open", axis=1, inplace=True)
#cluster.drop("Close", axis=1, inplace=True)


# ANALISI LONG

rule_long = (cluster.Close.shift(1) < cluster.Low.shift(2)) & \
            (cluster.Close.shift(2) < cluster.Low.shift(3)) & \
            (cluster.Close.shift(3) < cluster.Low.shift(4))

cluster["RuleLong"] = rule_long
cluster['RuleLongDecoded'] = np.where(cluster['RuleLong'] == True, 1, 0)

cluster['CloseOpenPercDecoded_Long'] = np.where(cluster['CloseOpenPerc'] > 0, 1, 0)

events_long = cluster[cluster.RuleLong == True].Open.count() # Open serve a restituire un unico valore e avremmo potuto usare qualsiasi campo!

events_long_OK = cluster.Open[(rule_long==True) & (cluster.CloseOpenPerc > 0)].count()

percent_win_long = round(events_long_OK / events_long * 100,2)

print("")
print("Regola a segno sul fronte Long per il", percent_win_long, "% dei campioni (", events_long_OK, "su", events_long,")")


# ANALISI SHORT

rule_short = (cluster.Close.shift(1) > cluster.High.shift(2)) & \
             (cluster.Close.shift(2) > cluster.High.shift(3)) & \
             (cluster.Close.shift(3) > cluster.High.shift(4))

cluster["RuleShort"] = rule_short
cluster['RuleShortDecoded'] = np.where(cluster['RuleShort'] == True, 1, 0)

cluster['CloseOpenPercDecoded_Short'] = np.where(cluster['CloseOpenPerc'] < 0, 1, 0)

events_short = cluster[cluster.RuleShort == True].Open.count() # Open serve a restituire un unico valore e avremmo potuto usare qualsiasi campo!

events_short_OK = cluster.Open[(rule_short==True) & (cluster.CloseOpenPerc < 0)].count()

percent_win_short = round(events_short_OK / events_short * 100,2)

print("")
print("Regola a segno sul fronte Short per il", percent_win_short, "% dei campioni(", events_short_OK, "su", events_short,")")

5.4.0


Il file contiene 68540 record

Regola a segno sul fronte Long per il 66.67 % dei campioni ( 22 su 33 )

Regola a segno sul fronte Short per il 57.89 % dei campioni( 22 su 38 )


Aggiungiamo un setup simmetrico sulla volatilità:

In [10]:
# ANALISI LONG

rule_long = (cluster.BodyRangePerc.shift(1) < 50) & \
            (cluster.Close.shift(1) < cluster.Low.shift(2)) & \
            (cluster.Close.shift(2) < cluster.Low.shift(3)) & \
            (cluster.Close.shift(3) < cluster.Low.shift(4))

cluster["RuleLong"] = rule_long
cluster['RuleLongDecoded'] = np.where(cluster['RuleLong'] == True, 1, 0)

cluster['CloseOpenPercDecoded_Long'] = np.where(cluster['CloseOpenPerc'] > 0, 1, 0)

events_long = cluster[cluster.RuleLong == True].Open.count() # Open serve a restituire un unico valore e avremmo potuto usare qualsiasi campo!

events_long_OK = cluster.Open[(rule_long==True) & (cluster.CloseOpenPerc > 0)].count()

percent_win_long = round(events_long_OK / events_long * 100,2)

print("")
print("Regola a segno sul fronte Long per il", percent_win_long, "% dei campioni (", events_long_OK, "su", events_long,")")


# ANALISI SHORT

rule_short = (cluster.BodyRangePerc.shift(1) < 50) & \
             (cluster.Close.shift(1) > cluster.High.shift(2)) & \
             (cluster.Close.shift(2) > cluster.High.shift(3)) & \
             (cluster.Close.shift(3) > cluster.High.shift(4))

cluster["RuleShort"] = rule_short
cluster['RuleShortDecoded'] = np.where(cluster['RuleShort'] == True, 1, 0)

cluster['CloseOpenPercDecoded_Short'] = np.where(cluster['CloseOpenPerc'] < 0, 1, 0)

events_short = cluster[cluster.RuleShort == True].Open.count() # Open serve a restituire un unico valore e avremmo potuto usare qualsiasi campo!

events_short_OK = cluster.Open[(rule_short==True) & (cluster.CloseOpenPerc < 0)].count()

percent_win_short = round(events_short_OK / events_short * 100,2)

print("")
print("Regola a segno sul fronte Short per il", percent_win_short, "% dei campioni(", events_short_OK, "su", events_short,")")


Regola a segno sul fronte Long per il 61.54 % dei campioni ( 8 su 13 )

Regola a segno sul fronte Short per il 63.16 % dei campioni( 12 su 19 )


# Ripetiamo l'analisi su periodicità weekly:

In [11]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__) # necessaria versione >= 1.9.0

import cufflinks as cf

# Per utilizzo con Notebooks
init_notebook_mode(connected=True)

# Per utilizzo offline
cf.go_offline()


filename = "@GC_60min.txt"
startDate = "20130101"
endDate = "20180810"
period = "W"

startDateParsed = datetime.date(int(startDate[0:4]), int(startDate[4:6]), int(startDate[6:]))
endDateParsed = datetime.date(int(endDate[0:4]), int(endDate[4:6]), int(endDate[6:]))

instrument = pd.read_csv(filename,
                         usecols=['Date','Time','Open','High','Low','Close','Up','Down'], 
                         parse_dates=[['Date', 'Time']])
print("Il file contiene", len(instrument), "record")
instrument.index = instrument['Date_Time']
instrument.drop('Date_Time',axis=1,inplace=True)
instrument['AvgPrice'] = round(instrument.iloc[:,0:4].mean(axis=1),2)
instrument['Range'] = round(instrument['High'] - instrument['Low'],2)
instrument['Body'] = abs(instrument['Open'] - instrument['Close'])
instrument['CloseOpenPerc'] = round((instrument['Close'] - instrument['Open']) / instrument['Open'] * 100,2)
instrument['BodyRangePerc'] = round(instrument['Body']/instrument['Range'] * 100,2)
# Percentuale della barra (all'interno del range) dove si attesta il close
instrument['CloseBarPerc'] = round((instrument['Close'] - instrument['Low']) * 100 / instrument['Range'],2)
instrument['DeltaClosePerc'] = round((instrument['Close'] - instrument['Close'].shift(1)) / instrument['Close'].shift(1) * 100,2)
instrument['DeltaAvgPricePerc'] = round((instrument['AvgPrice'] - instrument['AvgPrice'].shift(1)) / instrument['AvgPrice'].shift(1) * 100,2)

instrument = instrument.loc[startDateParsed:endDateParsed]
#instrument.head(10)

# creazione Pandas DataFrame di lavoro con i soli campi utili
cluster = pd.DataFrame(index=instrument.Open.resample(period).first().dropna().index.copy())
cluster["Open"] = instrument.Open.resample(period).first().dropna()
cluster["High"] = instrument.High.resample(period).max().dropna()
cluster["Low"] = instrument.Low.resample(period).min().dropna()
cluster["Close"] = instrument.Close.resample(period).last().dropna()
cluster['Range'] = round(cluster['High'] - cluster['Low'],2)
cluster['Body'] = abs(cluster['Open'] - cluster['Close'])
cluster['BodyRangePerc'] = round(cluster['Body']/cluster['Range'] * 100,2)
cluster['DeltaClosePerc'] = round((cluster['Close'] - cluster['Close'].shift(1)) / cluster['Close'].shift(1) * 100,2)
cluster['CloseOpenPerc'] = round((cluster['Close'] - cluster['Open']) / cluster['Open'] * 100,2)

cluster.head(10)

#cluster.drop("Open", axis=1, inplace=True)
#cluster.drop("Close", axis=1, inplace=True)


# ANALISI LONG

rule_long = (cluster.BodyRangePerc.shift(1) < 50) & \
            (cluster.Close.shift(1) < cluster.Low.shift(2)) & \
            (cluster.Close.shift(2) < cluster.Low.shift(3)) & \
            (cluster.Close.shift(3) < cluster.Low.shift(4))

cluster["RuleLong"] = rule_long
cluster['RuleLongDecoded'] = np.where(cluster['RuleLong'] == True, 1, 0)

cluster['CloseOpenPercDecoded_Long'] = np.where(cluster['CloseOpenPerc'] > 0, 1, 0)

events_long = cluster[cluster.RuleLong == True].Open.count() # Open serve a restituire un unico valore e avremmo potuto usare qualsiasi campo!

events_long_OK = cluster.Open[(rule_long==True) & (cluster.CloseOpenPerc > 0)].count()

percent_win_long = round(events_long_OK / events_long * 100,2)

print("")
print("Regola a segno sul fronte Long per il", percent_win_long, "% dei campioni (", events_long_OK, "su", events_long,")")


# ANALISI SHORT

rule_short = (cluster.BodyRangePerc.shift(1) < 50) & \
             (cluster.Close.shift(1) > cluster.High.shift(2)) & \
             (cluster.Close.shift(2) > cluster.High.shift(3)) & \
             (cluster.Close.shift(3) > cluster.High.shift(4))

cluster["RuleShort"] = rule_short
cluster['RuleShortDecoded'] = np.where(cluster['RuleShort'] == True, 1, 0)

cluster['CloseOpenPercDecoded_Short'] = np.where(cluster['CloseOpenPerc'] < 0, 1, 0)

events_short = cluster[cluster.RuleShort == True].Open.count() # Open serve a restituire un unico valore e avremmo potuto usare qualsiasi campo!

events_short_OK = cluster.Open[(rule_short==True) & (cluster.CloseOpenPerc < 0)].count()

percent_win_short = round(events_short_OK / events_short * 100,2)

print("")
print("Regola a segno sul fronte Short per il", percent_win_short, "% dei campioni(", events_short_OK, "su", events_short,")")

5.4.0


Il file contiene 68540 record

Regola a segno sul fronte Long per il 66.67 % dei campioni ( 2 su 3 )

Regola a segno sul fronte Short per il 66.67 % dei campioni( 2 su 3 )


### Ancora non stiamo parlando di Trading System ma di semplici test sui setup!